In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from time import sleep

import twitter, re, datetime, pandas as pd
import sys
import re
import time
import tweepy
from pprint import pprint 

import os

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle 

# get weather data

## Statics

In [3]:
import requests, json

In [5]:
def kelvin_converter(deg_k):
    
    return (deg_k - 273.15) * 9/5 + 32

In [55]:
weather_data_path = "http://storage.googleapis.com/twitter-weather/weather-data/"

with open('./info/weather_key.txt', "r") as file:
    api_key_weather = file.read().replace('\n', '')
    
# https://www.geeksforgeeks.org/python-find-current-weather-of-any-city-using-openweathermap-api/

base_url = "http://api.openweathermap.org/data/2.5/weather?"
city_name = "Austin"
complete_url = base_url + "appid=" + api_key_weather + "&q=" + city_name 

## Recurring

In [56]:
response = requests.get(complete_url) 
x = response.json() 

In [57]:
# declare vars
weather_main = None
weather_description = None
temp_K = None
temp_F = None
pressure = None
humidity = None
temp_min = None
temp_max = None
sea_level = None
grnd_level = None
visibility = None
wind_speed = None
wind_deg = None
rain_type = None
rain_list = None
clouds = None
city_name = None

In [58]:
try:
    city_name = x['name']
except:
    print('bad var city')
try:
    weather_main = x['weather'][0]['main']
except:
    print('bad var weather_main')
    
try:
    weather_description = x['weather'][0]['description']
except:
    print('bad var weather_description')
    
try:
    temp_K = x['main']['temp']
    temp_F = kelvin_converter(temp_K)
except:
    print('bad var temp_K')
    
try:
    pressure = x['main']['pressure']
except:
    print('bad var pressure')
        
try:
    humidity = x['main']['humidity']
except:
    print('bad var humidity')
        
try:
    temp_min = x['main']['temp_min']
except:
    print('bad var temp_min')
        
try:
    temp_max = x['main']['temp_max']
except:
    print('bad var temp_max')
        
try:
    sea_level = x['main']['sea_level']
except:
    print('bad var sea_level')
        
try:
    grnd_level = x['main']['grnd_level']
except:
    print('bad var grnd_level')
        
try:
    visibility = x['visibility']
except:
    print('bad var visibility')
        
try:
    wind_speed = x['wind']['speed']
except:
    print('bad var wind_speed')
        
try:
    wind_deg = x['wind']['deg']
except:
    print('bad var wind_deg')
        
try:
    rain_type = list(x['rain'].keys())[0]
except:
    print('bad var rain_type')
        
try:
    rain_list = x['rain'][rain_type]
except:
    print('bad var rain_list')
        
try:
    clouds = x['clouds']['all']
except:
    print('bad var clouds')
        

bad var sea_level
bad var grnd_level
bad var rain_type
bad var rain_list


In [59]:
timestamp = datetime.datetime.now()

In [60]:
data_dict = {
    'timestamp': timestamp, 
    'weather_main' :weather_main, 
    'weather_description'  :weather_description ,
    'temp_K'  :temp_K ,
    'temp_F'  :temp_F ,
    'pressure'  :pressure ,
    'humidity'  : humidity,
    'temp_min'  :temp_min ,
    'temp_max'  :temp_max ,
    'sea_level' : sea_level,
    'grnd_level' : grnd_level,
    'visibility' : visibility,
    'wind_speed' :wind_speed ,
    'wind_deg' : wind_deg,
    'rain_type': rain_type,  
    'rain_list': rain_list, 
    'clouds':clouds , 
    'city_name':city_name, 
}

In [61]:
# pd.DataFrame(data_dict, index=[0]).to_csv('./info/weather_start.csv', index=False)

In [62]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
            './info/log_queries.json',
            scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [65]:
q_base = """
INSERT INTO metridaticsmain.webscrapes.weather_data (
timestamp,
weather_main,
weather_description,
temp_K,
temp_F,
pressure,
humidity,
temp_min,
temp_max,
sea_level,
grnd_level,
visibility,
wind_speed,
wind_deg,
rain_type,
rain_list,
clouds,
city_name
)
 
VALUES ('{}', '{}', '{}', {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, '{}', {}, {}, '{}')


;
""".format(timestamp,
weather_main,
weather_description,
temp_K,
temp_F,
pressure,
humidity,
temp_min,
temp_max,
sea_level,
grnd_level,
visibility,
wind_speed,
wind_deg,
rain_type,
rain_list,
clouds,
city_name).replace('\n', '').replace("None","NULL").replace("'NULL'", "NULL")

In [67]:
query_job = client.query(q_base, location="US")
query_job.result()

# Twitter

# Get New Data (Script Needs)

In [89]:
# One time
with open('./info/twitterapi2.txt') as f:
    ck, cs, atk, ats = f.read().split(',')

In [101]:
# your keys go here:
twitter_keys = {
    'consumer_key':        ck.replace('\n', ''),
    'consumer_secret':     cs.replace('\n', ''),
    'access_token_key':    atk.replace('\n', ''),
    'access_token_secret': ats.replace('\n', '')
}

In [107]:
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [95]:
# Old version
# update_interval_hours = 6
# for i in range(24):
#     sleep(update_interval_hours*60*60)

# # Each Time
# end_date = '{}-{}-{}'.format(tomorrow.year, tomorrow.month, tomorrow.day)   
# start_date = '{}-{}-{}'.format(yesterday.year, yesterday.month, yesterday.day)

# Otherwise, won't update public doc for 1 hour
# !gsutil -h "Cache-Control:no-cache, max-age=0" cp -a public-read myfile.json gs://mybucket

In [96]:
time_now = datetime.datetime.now()

In [118]:
time_start = datetime.datetime.now() - datetime.timedelta(days=1)
time_stop = datetime.datetime.now() + datetime.timedelta(days=1)

In [119]:
# Each Time
start_date = '{}-{}-{}'.format(time_start.year, time_start.month, time_start.day)
end_date = '{}-{}-{}'.format(time_stop.year, time_stop.month, time_stop.day)   

In [120]:
create_big = True
if create_big:
    df_big = pd.DataFrame()
    places = ["Austin, TX", "Portland, OR"]

else:
    df_small = pd.DataFrame()
#     places = ["Austin, TX"]
    places = ["Austin, TX"]
tweetsPerQry = 100
counter = 0

for place_q in places: 

    place = api.geo_search(query=place_q, granularity="city")
    place_id = place[0].id

    max_id = -1
    for _ in range(20):
        try:
            if (max_id <= 0):
                new_tweets = api.search(q='place:%s' % place_id, count=tweetsPerQry, since=start_date, until=end_date)
            else:
                new_tweets = api.search(q='place:%s' % place_id, count=tweetsPerQry, max_id=str(max_id - 1), since=start_date, until=end_date)

            df_text = pd.DataFrame([new_tweets[i]._json['text'] for i in range(len(new_tweets))], columns=['tweet_text'])
            df_text['tweet_time']=[new_tweets[i]._json['created_at'] for i in range(len(new_tweets))]
            df_text['tweet_place'] = place_q
            if create_big:
                df_big = pd.concat([df_big, df_text])
            else:
                df_small = pd.concat([df_small, df_text])
            
            if not new_tweets:
                counter += 1
#                 if create_big:
#                     df_big.to_csv('./collected_tweets_{}_{}.csv'.format(start_date, counter)) 
#                 else: 
#                     df_small.to_csv('./collected_tweets_{}_{}.csv'.format(start_date, counter)) # Should provide log of last good pull at least
                    
                print("No more tweets found")
                break

            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            print('    all_done')
            break
        sleep(5)

No more tweets found


KeyboardInterrupt: 

In [126]:
# q_base = """
# CREATE TABLE metridaticsmain.webscrapes.twitter_data_raw (
#  tweet_time timestamp,
#  tweet_text STRING,
#  tweet_place STRING

# );
# """

# query_job = client.query(q_base, location="US")
# query_job.result()

In [4]:
import numpy as np

In [210]:
df_big['tweet_time'] = pd.to_datetime(df_big.tweet_time)

In [169]:
def clean_chars(val_string):
    val_string = val_string.replace("'", "")
    val_string = val_string.replace("\\", "")
    
    return val_string

In [212]:
def combine_tweet_info(tweet_text, tweet_time, tweet_place):
    tweet_text = clean_chars(tweet_text)
    
    return "('" + tweet_place + "', '" + str(tweet_time) + "', '" + tweet_text + "') "

In [226]:
values_combined = df_big[df_big.tweet_place=='Austin, TX'].iloc[:500,:].apply(lambda x: combine_tweet_info(x['tweet_text'], x['tweet_time'], x['tweet_place'] ), axis=1).values

In [227]:
values_combined = ', '.join(values_combined)

In [228]:
q_base = """
INSERT INTO metridaticsmain.webscrapes.twitter_data_raw (
 tweet_place,
 tweet_time,
 tweet_text


)

 VALUES {}
;
""".format(values_combined)
q_base = q_base.replace("\n", "")

In [229]:
query_job = client.query(q_base, location="US")
query_job.result()

In [230]:
values_combined[:100]

"('Austin, TX', '2019-09-27 03:06:09+00:00', 'Omg Jordan Howard have a game! #PHIvGB') , ('Austin, TX"

In [17]:
# testing scripts

In [1]:
import os, sys
cwd = os.getcwd()
sys.path.append(cwd+'/scripts/')

In [18]:
from importlib import import_module
import_module("my_bigq")
import_module("run_weather")
from my_bigq import bigquery_handler 
import run_weather as rw
import run_twitter as rt

In [4]:
_test = rw.run_city("Austin")

bad var sea_level
bad var grnd_level
bad var rain_type
bad var rain_list


In [9]:
x = rw.get_weather("Austin")

In [10]:
data_dict = rw.process_weather(x)

bad var sea_level
bad var grnd_level
bad var rain_type
bad var rain_list


In [14]:
_test2 = bigquery_handler(q_base)

In [15]:
_test3 = _test2.run_query()

In [16]:
_test3

In [19]:
api = rt.define_api()

In [21]:
df_big = rt.get_twitter(api, "Austin, TX")

In [36]:
df_big.shape[0]

612

In [24]:
from importlib import reload

In [33]:
reload(rt)

<module 'run_twitter' from '/home/conor/projects/is_it_raining/scripts/run_twitter.py'>

In [82]:
1%24

1

In [34]:
values_combined = rt.process_twitter(df_big)

found 1 chunks


In [38]:
from numpy import floor

In [41]:
def clean_chars(val_string):
    val_string = val_string.replace("'", "")
    val_string = val_string.replace("\\", "")

    return val_string

def combine_tweet_info(tweet_text, tweet_time, tweet_place):
    tweet_text = clean_chars(tweet_text)

    return "('" + tweet_place + "', '" + str(tweet_time) + "', '" + tweet_text + "') "


In [77]:
num_rows = df_big.shape[0]
values_combined = []
n_chunks = int(floor(num_rows/500))
print('found {} chunks'.format(n_chunks))

for n in range(n_chunks+1):
    _rel = df_big.iloc[n*500: (n+1)*500, :]
    if _rel.shape[0] == 0: # cases where df_big is a multiple of 1000
        continue
    _rel_combined = _rel.apply(lambda x: combine_tweet_info(x['tweet_text'], x['tweet_time'], x['tweet_place'] ), axis=1).values
    values_combined.append(_rel_combined)

for i in range(len(values_combined)):
    values_combined[i] = ', '.join(values_combined[i])

found 1 chunks


In [80]:
rt.send_to_table(values_combined)

In [ ]:
# df_big.to_csv('./5_cities_2019.1.17.csv', index=False)

In [11]:
model_columns = cvec.get_feature_names()
df_small = pd.concat([df_small.reset_index().drop('index', axis=1), pd.DataFrame(cvec.transform(df_small.tweet_text.str.lower()).todense(), columns=cvec.get_feature_names())], axis=1)
_X = df_small.loc[:, [col for col in df_small.columns if col in model_columns]]

_X.fillna(0, inplace=True)

df_small['predicted'] = lr.predict(_X)
df_small['probas'] = [element[1] for element in lr.predict_proba(_X)]

In [12]:
excludes = ['tweet_text', 'tweet_time', 'tweet_place', 'is_rain', 'predicted']
_rain_probability = df_small.loc[:, excludes + ['predicted', 'probas']].groupby('tweet_place').mean()['probas'].values[0]
_rain_probability = round(_rain_probability*100, 1)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


# Refit model here

In [2]:
cvec = pickle.load(open('vectorizer.sav', 'rb')) 
lr = pickle.load(open('lin_regressor.sav', 'rb')) 

In [3]:
df_big=pd.concat([
    pd.read_csv('./data/5_cities_2019.1.3.csv'), 
    pd.read_csv('./data/5_cities_2019.1.17.csv'), 
#     pd.read_csv('./data/austin_2019_02_19.csv')
])
df_big.index = range(df_big.shape[0])
df_big["tweet_time"] = pd.to_datetime(df_big.tweet_time)

In [6]:
df_big.groupby(['tweet_place', df_big.tweet_time.dt.date]).count().loc[:, ["tweet_text"]]

tweet_text
tweet_place       tweet_time            
Austin, TX        2019-01-03         358
                  2019-01-17         788
Boston, MA        2019-01-03          82
                  2019-01-17         625
Portland, OR      2019-01-03         442
                  2019-01-17         654
San Francisco, CA 2019-01-03         316
                  2019-01-17         654
Shreveport, LA    2019-01-03         455
                  2019-01-17         474

In [23]:
cvec = CountVectorizer()

df_big=df_big.join(pd.DataFrame(cvec.fit_transform(df_big.tweet_text.str.lower()).todense(), columns=cvec.get_feature_names()))

In [24]:
df_big["tweet_time"] = pd.to_datetime(df_big.tweet_time)
df_big['is_rain']= (((df_big.tweet_place == "Shreveport, LA") & (df_big.tweet_time.dt.date == datetime.date(2019, 1, 3))) |\
                    ((df_big.tweet_place == "San Francisco, CA") & (df_big.tweet_time.dt.date == datetime.date(2019, 1, 17)))|
                    ((df_big.tweet_place == "Austin, TX") & (df_big.tweet_time.dt.date == datetime.date(2019, 2, 20)))
                   )*1

In [25]:
excludes = ['tweet_text', 'tweet_time', 'tweet_place', 'is_rain', 'predicted']

In [26]:
X = df_big.loc[:, [col for col in df_big.columns if col not in excludes]]
y = df_big.is_rain

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

df_big['predicted'] = lr.predict(X)
df_big['probas'] = [element[1] for element in lr.predict_proba(X)]

df_big.loc[:, excludes + ['predicted', 'probas']].groupby('tweet_place').mean()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6125721616420783


,is_rain,predicted,predicted,probas
tweet_place,,,,
"Austin, TX",0.547393,0.467615,0.467615,0.463345
"Boston, MA",0.000000,0.052334,0.052334,0.224023
"Portland, OR",0.000000,0.043796,0.043796,0.200735
"San Francisco, CA",0.674227,0.555670,0.555670,0.516902
"Shreveport, LA",0.489774,0.392896,0.392896,0.432043


In [31]:
df_big[(df_big.tweet_place=="Austin, TX")&(df_big.tweet_time.dt.date>datetime.date(2019,2,1))].loc[:, 'probas'].mean()

0.648584596895982

In [32]:
# Doesn't seem to help too much filter 
df_big[(df_big.tweet_place=="Austin, TX")&(df_big.tweet_time.dt.date>=datetime.date(2019,2,17))]\
.loc[:, ['tweet_time','probas']].sort_values(by='tweet_time').tail(20).mean()

probas    0.646408
dtype: float64

In [33]:
# pickle.dump(cvec, open('vectorizer.sav', 'wb')) 
# pickle.dump(lr, open('lin_regressor.sav', 'wb')) 

In [17]:
cvec = pickle.load(open('vectorizer.sav', 'rb')) 
lr = pickle.load(open('lin_regressor.sav', 'rb')) 

In [39]:
# check small frame
df_small = pd.read_csv('./data/austin_2019-01-27_to_2019-01-28.csv')
df_small["tweet_time"] = pd.to_datetime(df_small.tweet_time)

In [60]:
for i in range(1,10):
    print('{} to {} hours ago had {} tweets'.format(i , i+1, 
                                                   df_small[(df_small.tweet_time >= (datetime.datetime.now() - datetime.timedelta(hours=i+1)))&\
                                                          ( df_small.tweet_time <= (datetime.datetime.now() - datetime.timedelta(hours=i)))].shape[0]))

1 to 2 hours ago had 363 tweets
2 to 3 hours ago had 291 tweets
3 to 4 hours ago had 190 tweets
4 to 5 hours ago had 120 tweets
5 to 6 hours ago had 57 tweets
6 to 7 hours ago had 53 tweets
7 to 8 hours ago had 77 tweets
8 to 9 hours ago had 81 tweets
9 to 10 hours ago had 67 tweets


In [42]:
# df_small = df_small[df_small.tweet_time > (datetime.datetime.now() - datetime.timedelta(hours=4))] # Expand here to 6 if shape[0], then allow all
df_small = pd.concat([df_small, pd.DataFrame(cvec.transform(df_small.tweet_text.str.lower()).todense(), columns=cvec.get_feature_names())], axis=1)

In [43]:
_X = df_small.loc[:, [col for col in df_small.columns if col in X.columns]]

In [45]:
_X.fillna(0, inplace=True)

df_small['predicted'] = lr.predict(_X)
df_small['probas'] = [element[1] for element in lr.predict_proba(_X)]

df_small.loc[:, excludes + ['predicted', 'probas']].groupby('tweet_place').mean()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,is_rain,predicted,predicted,probas
tweet_place,,,,
"Austin, TX",NaN,0.254275,0.254275,0.395661
